## Imports

In [915]:
from scipy.linalg import solve_banded
from scipy.interpolate import interp1d
import numpy as np
from tqdm.notebook import tqdm #if you dont have tqdm type in your terminal: conda install -c conda-forge tqdm
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
import matplotlib.cm as cm
import time
from scipy import signal
from scipy.special import comb

%matplotlib inline

## Diffusion model and helper functions

In [535]:
class Diffuse1D:
    def __init__(self, length, spacing, tstep, left, right, K, sed_Q, no_flux_boundary=False):
        self.x = np.arange(0, length, spacing)
        self.u = (
            left - self.x * (left - right) / length
        )  # sets initial to steady state solution
        N = self.x.size
        self.K = K
        self.dt = tstep
        self.dx = spacing
        self.time = 0
        self.base_level_fun = interp1d([0, 1], [0, 0], bounds_error=False, fill_value=0)
        self.base_level = self.base_level_fun(self.time)
        self.sed_Q = sed_Q 
        self.left = left
        self.right = right
        self.r = self.K * (self.dt / (2 * self.dx ** 2))
        self.no_flux_boundary = no_flux_boundary
        
        # initialize matrices A, B and b array
        self.b = np.zeros((N))
        self.b[0] = self.left * 2 * self.r
        self.b[-1] = self.right * 2 * self.r
        self.A = np.zeros((N, N))
        self.B = np.zeros((N, N))

        self.make_B(self.B, self.r)
        self.A, self.A_inv = self.make_A(self.A, self.r)

        self.coastline = 0
        self.Q = np.zeros((N))
        self.update_coastline()


    def make_A(self, A, r):
        l = [-r for i in range(1, A.shape[0] - 2)]
        c = [1 + 2 * r for i in range(A.shape[0] - 2)]
        u = [-r for i in range(A.shape[0] - 3)]
        np.fill_diagonal(A[1:], u)
        np.fill_diagonal(A[:, 1:], l)
        np.fill_diagonal(A, c)
        A_inv = np.linalg.inv(A)
        return A, A_inv

    def make_B(self, B, r):
        l = [r for i in range(1, B.shape[0] - 2)]
        c = [1 - 2 * r for i in range(B.shape[0] - 2)]
        u = [r for i in range(B.shape[0] - 3)]
        np.fill_diagonal(B[1:], u)
        np.fill_diagonal(B[:, 1:], l)
        np.fill_diagonal(B, c)
        return B

    def set_baselevel(self, time, rsl):
        # can pass a set of paired time/rsl and model will interpolate for each dt
        self.base_level_fun = interp1d(time, rsl)

    def run_step(self):
        if self.no_flux_boundary:
            self.b[0] = self.u[0] * 2 * self.r
            self.b[-1] = self.u[-1] * 2 * self.r
        #         self.update_K()  # can disable if K is not changing with each t_step (ie as a function of elevation)
        self.Q *= 0  # clearing any old sed flux terms
        self.base_level = self.base_level_fun(self.time)  # update base level
        if self.base_level>np.min(self.u):
            self.update_coastline()  # select coastline point for sed flux
            self.Q[self.coastline] += self.sed_Q/self.dx  # add sediment to the coastline
        self.bb = (
            self.B.dot(self.u) + self.b + self.dt * self.Q
        )  # matrix addition
        self.u = self.A_inv.dot(self.bb)
        self.time += self.dt  # increment timestep        

    def update_coastline(self):
        # finds the first grid point below base level starting on the left hand side
        try:
            self.coastline = np.where(self.u < (self.base_level))[0][0]
        except:
            self.coastline = 1


def sawtooth_wave(n, x):
    y = np.zeros_like(x)
    for k in range(1, n + 1):
        y += (comb(2*n, n-k) / comb(2*n, n)) * ( np.sin(k*x) / k )
    return y


def get_strat_column(beds, age, rsl, loc, skip=1):
    facies_limits = [0.5, -0.05, -.5, -1, -2, -2000]
    facies_w = [a for a in [0.8, 0.6, 0.5, 0.4, 0.3, 0.2]]
    facies_c = [
        "#ffb142",  # 
        "#33d9b2",  # 
        "#34ace0",  # 
        "#706fd3",  # 
        "#2c2c54",  # 
        "#84817a",  #
    ]   

    strat = np.array(beds.copy())[::skip][:, loc]
    time_strat = np.array(age.copy())[::skip]
    rsl_strat = np.array(rsl.copy())[::skip]

    for i in range(0, strat.size):
        older = strat[:-i]
        older_rsl = rsl_strat[:-i]
        older[older > strat[-i]] = strat[-i]
        older_rsl[older > strat[-i]] = strat[-i]
        strat[:-i] = older
        rsl_strat[:-i] = older_rsl
        rsl_strat = np.array(rsl.copy())[::skip]

    thicknesses = np.zeros(strat.shape[0])
    heights = np.zeros(strat.shape[0])
    
    water_depths = rsl_strat-strat
    facies = np.zeros(water_depths.size)
    colors = np.zeros(water_depths.size).astype(str)
    water_depths = -1 * water_depths

    for j in range(len(facies_limits) - 1, -1, -1):
        facies[water_depths > facies_limits[j]] = facies_w[j]
        colors[water_depths > facies_limits[j]] = facies_c[j]

        bed_facies = []
        bed_top = []
        bed_colors = []

        changes = np.where(np.diff(facies) != 0)[0]
        for c in [*changes, len(facies) - 1]:
            bed_facies += [facies[c]]
            bed_top += [strat[c]]
            bed_colors += [colors[c]]

        h = np.diff([np.min(strat), *bed_top])
        bed_bottom=bed_top-h
        
    return (np.array(bed_facies).ravel(), 
            np.array(bed_bottom).ravel(), 
            np.array(h).ravel(), 
            np.array(bed_colors).ravel())

def plot_column(bed_facies,bed_bottom,h,bed_colors,left=0):
    plt.figure(figsize=(3,12))
    plt.barh(
        bed_bottom,
        bed_facies,
        height=h,
        left=left,
        color=bed_colors,
        align="edge",
        lw=1,
        edgecolor=(0.2, 0.2, 0.2),zorder=4
    )

## Initial conditions

In [906]:
dt = 1  # timestep
base_level_rise = 100  # long term subsidence in meters
dx = 10  # x grid spacing
total_time = 3e6  # duration of simulation in years
initial_baselevel = 1  # in meters
sed_Q = 0.1  # sedimentation flux in m/2 (dx is handled inside the model)

#create an instance of Diffuse1D (defined above)
model = Diffuse1D(
    length=1000,
    spacing=dx,
    tstep=dt,
    left=0,
    right=0,
    K=2e-2,
    sed_Q=sed_Q,
    no_flux_boundary=True,
)

xt = np.linspace(0, total_time, 10000)  # creating uniform timegrid
RSL1 = -1.5 * np.sin(xt * 2 * np.pi * (1 / 20000))  # cyclic sea level component 1
RSL2 = 1 * sawtooth_wave(5, xt * 2 * np.pi * (1 / 100000))  # cyclic sea level component 2
RSL = (RSL1 + RSL2 + base_level_rise / (total_time) * xt + initial_baselevel)  # cyclic sea level + subsidence
# creates a function in the Diffuse1D model mapping your sea level boundary condition to time
model.set_baselevel(xt, RSL)

Plotting your model sea level boundary condition

In [ ]:
#to plot your model sea level boundary condition
plt.plot(xt/1000,model.base_level_fun(xt))
plt.gca().set_xlabel('Time (kilo-years)')
plt.gca().set_ylabel('Base level')

Running your model

In [ ]:
# initial lists to store model outputs throughout the simulation
beds = [] #model topography
age = [] #model time
rsl = [] #relative (local) sea level
progress_bar = tqdm(range(int(total_time / dt / 1))) #run the model for the full duration, the tqdm wrapper provides a progress bar
for i in progress_bar:
    model.run_step() #run 1 timestep dt
    if i%50==0: #save every 50 steps to our lists
        beds.append(model.u)
        age.append(model.time)
        rsl.append(model.base_level)

Extracting stratigraphic columns from your model

In [ ]:
#this cell shows how to use functions get_strat_column to extract and plot a stratigraphic column from your model output
column_number = 10
bed_facies, bed_bottom, bed_thickness, bed_colors = get_strat_column(beds, age, rsl, column_number, skip=10)
plot_column(bed_facies, bed_bottom, bed_thickness, bed_colors, left=column_number)
plt.gca().set_aspect(.05)

Creating a time series of parasequence thicknesses from your stratigraphic column

In [ ]:
#this cell shows how to extract and plot a time series of parasequence thicknesses from your stratigraphic column
bed_tops = bed_bottom + bed_thickness
para_tops = np.insert(bed_tops[np.where(np.diff(bed_facies) < 0)], 0, 0)
para_thickness = np.diff(para_tops)
para_thickness = para_thickness[para_thickness > 0.1]  # only preserve greater than 10 cm
plt.plot(para_thickness)
plt.gca().set_xlim(0, 100)

The cell below shows how to plot the model beds, after accounting for erosion

In [ ]:
skip = 100 #calculate/plot every 100 beds to save time, a little less accurate
beds_eroded = np.copy(np.array(beds)[::skip,:])
for i in range(0,np.array(beds_eroded).shape[0]-1):
    beds_eroded[i,:] = np.min(beds_eroded[i+1:,:],axis=0)
    
for i in range(beds_eroded.shape[0]):
    plt.plot(beds_eroded[i,:],color='k',alpha=.1,lw=1)